In [ ]:
# This program is used to train the Gate Recurrent Neural Network 
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
import glob,os
import math
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor    
torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)


In [ ]:
#read the recording data from the folder
base_path = ' ' # enter your local path
for k in range (65,92):
    #alphabet = ord(alphabet)-97
    a = np.load(base_path+'/'+chr(k)+chr(k)+'.npy')
    for ax in range(3):
        for i in range(5,100): # how many times you record
            for j in range(400): # time points in one trial
                temp = j
                while (a[i][ax][j]<-60): # arduino sometimes pass 0 to the computer, as a result, use previous data to replace 0
                    temp = j-1
                    a[i][ax][j] = a[i][ax][temp]
    for ax in range(3): # min max normalization
        for i in range(5,100):
            min = np.mean(a[i][ax])
            max = np.max(a[i][ax])
            a[i][ax] = (a[i][ax]-min )/(max-min)
    np.save(base_path+'/'+chr(k)+'_r',a)
#plot x y z data
for k in range(len(thre)):
    for i in range(15,45):
        plt.plot(a[i][0][thre[i]:thre[i]+195])
plt.show()
for k in range(len(thre)):
    for i in range(15,45):
        plt.plot(a[i][1][thre[i]:thre[i]+195])
plt.show()
for k in range(len(thre)):
    for i in range(15,45):
        plt.plot(a[i][2][thre[i]:thre[i]+195])
plt.show()

In [ ]:
#read train dataset
train_dir = base_path
train_names = glob.glob(os.path.join(train_dir, '*') )
train_numpy = []
train_name = []
for image_path in train_names:
    #print(image_path)
    alphabet = image_path[-7]
    alphabet = ord(alphabet)-65
    alp_arr = [alphabet] * 95
    im = np.load(image_path)
    train_numpy.append(im[5:])
    train_name.append(alp_arr)
    print(image_path[-7],alphabet,len(train_numpy))
train_numpy = np.array(train_numpy).reshape(([-1,4,400])).astype('float')
train_numpy = np.transpose(train_numpy, (0, 2, 1))
train_name = np.array(train_name).reshape((-1))
np.save('train',train_numpy)
np.save('train_alphabet',train_name)
print(train_numpy.shape)
print(train_name.shape)

In [ ]:
#Min max normalizaion
for i in range(len(train_numpy)):
    for k in range(3):
        max_num = np.max(train_numpy[i,:,k])
        min_num = np.min(train_numpy[i,:,k])
        if((max_num - min_num)!= 0):
            train_numpy[i,:,k] = (train_numpy[i,:,k]-min_num)/(max_num - min_num)
        else:
            train_numpy[i,:,k] = 0

In [ ]:
#Pytorch dataset
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors,alphabet):
        self.tensors = tensors
        self.alphabet = alphabet
        
    def __getitem__(self, index):
        x = self.tensors[index]
        alph = self.alphabet[index]
        return x,alph.float() 

    def __len__(self):
        return len(self.tensors)


In [ ]:
#Dataloader
batch_size = 64
data = torch.from_numpy(train_numpy)
name = torch.from_numpy(train_name)
full_dataset = CustomTensorDataset(data,name)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
test_sampler = RandomSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

## GRU cell implmentation

In [ ]:
class GRUCell(nn.Module):

    """
    An implementation of GRUCell.

    """

    def __init__(self, input_size, hidden_size, bias=True):
        super(GRUCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 3 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 3 * hidden_size, bias=bias)
        self.reset_parameters()



    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)
    
    def forward(self, x, hidden):
        
        x = x.view(-1, x.size(1))
        
        gate_x = self.x2h(x) 
        gate_h = self.h2h(hidden)
        
        gate_x = gate_x.squeeze()
        gate_h = gate_h.squeeze()
        if(len(gate_x.size())<=1):
            gate_x = gate_x.unsqueeze(0)
            gate_h = gate_h.unsqueeze(0)             
        i_r, i_i, i_n = gate_x.chunk(3, 1)
        h_r, h_i, h_n = gate_h.chunk(3, 1)
        
        
        resetgate = F.sigmoid(i_r + h_r)
        inputgate = F.sigmoid(i_i + h_i)
        newgate = F.tanh(i_n + (resetgate * h_n))
        
        hy = newgate + inputgate * (hidden - newgate)
        
        
        return hy


In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(GRUModel, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim
         
        # Number of hidden layers
        self.layer_dim = layer_dim
         
       
        self.gru_cell = GRUCell(input_dim, hidden_dim, layer_dim)
        
        
        self.fc = nn.Linear(hidden_dim, output_dim)
     
    
    
    def forward(self, x):
        

        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
         
       
        outs = []
        
        hn = h0[0,:,:]
        
        for seq in range(x.size(1)):
            hn = self.gru_cell(x[:,seq,:], hn) 
            outs.append(hn)
            

        out = outs[-1].squeeze()
        
        out = self.fc(out) 
        # out.size() --> 100, 10
        return out
 

In [ ]:

input_dim = 4
hidden_dim = 128
layer_dim = 1  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 26
model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)
if torch.cuda.is_available():
    model.cuda()
     
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  


In [ ]:
from torchvision import models
from torchsummary import summary
summary(model, (400, 4))

In [ ]:

# Number of steps to unroll
seq_dim = 400 #time points of each alphabet
batch_size = 32
num_epochs = 6000 
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(train_dataloader):
          
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda().float())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim).float() )
            labels = Variable(labels)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.to(torch.int64))

        if torch.cuda.is_available():
            loss.cuda()

        loss.backward()
        optimizer.step()
        
        loss_list.append(loss.item())
        iter += 1
         
        if iter % 1 == 0:      
            correct = 0
            total = 0
            for (images, labels) in test_dataloader:
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda().float())
                else:
                    images = Variable(images.view(-1 , seq_dim, input_dim).float())
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
             
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

In [ ]:
torch.save(model, 'all_final.pt')
torch.save(model.state_dict(), 'all_weight.pt')